In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from geopy.geocoders import Nominatim
from math import radians, cos, sin, asin, sqrt
from numpy import cos, sin, arcsin, sqrt
from ast import literal_eval
from tqdm import tqdm
import requests
import json
import networkx as nx
import osmnx as ox
import requests
from geopy import Point
from geopy.distance import geodesic
import numpy as np

In [2]:
df_building = pd.read_csv('BeijingBuildingsFinal.csv')

In [3]:
df_building.rename(columns={'name' : 'Building'}, inplace = True)

In [4]:
df_civic = pd.read_csv('CivicDistance.csv')
df_fire = pd.read_csv('FireDistance.csv')
df_government = pd.read_csv('GovernmentDistance.csv')
df_hospital = pd.read_csv('HospitalDistance.csv')
df_public = pd.read_csv('PublicDistance.csv')
df_school = pd.read_csv('SchoolDistance.csv')
df_sports = pd.read_csv('SportsDistance.csv')
df_toilet = pd.read_csv('ToiletDistance.csv')
df_transport = pd.read_csv('TransportDistance.csv')

In [5]:
df_civic = df_civic.groupby('InputID').agg(lambda x: list(x))
df_fire = df_fire.groupby('InputID').agg(lambda x: list(x))
df_government = df_government.groupby('InputID').agg(lambda x: list(x))
df_hospital = df_hospital.groupby('InputID').agg(lambda x: list(x))
df_public = df_public.groupby('InputID').agg(lambda x: list(x))
df_school = df_school.groupby('InputID').agg(lambda x: list(x))
df_sports = df_sports.groupby('InputID').agg(lambda x: list(x))
df_toilet = df_toilet.groupby('InputID').agg(lambda x: list(x))
df_transport = df_transport.groupby('InputID').agg(lambda x: list(x))

In [6]:
df_civic = df_civic.reset_index()
df_fire = df_fire.reset_index()
df_government = df_government.reset_index()
df_hospital = df_hospital.reset_index()
df_public = df_public.reset_index()
df_school = df_school.reset_index()
df_sports = df_sports.reset_index()
df_toilet = df_toilet.reset_index()
df_transport = df_transport.reset_index()

In [7]:
df_civic.drop(columns=['TargetID'], inplace = True)
df_fire.drop(columns=['TargetID'], inplace = True)
df_government.drop(columns=['TargetID'], inplace = True)
df_hospital.drop(columns=['TargetID'], inplace = True)
df_public.drop(columns=['TargetID'], inplace = True)
df_school.drop(columns=['TargetID'], inplace = True)
df_sports.drop(columns=['TargetID'], inplace = True)
df_toilet.drop(columns=['TargetID'], inplace = True)
df_transport.drop(columns=['TargetID'], inplace = True)

In [ ]:
df_civic
df_fire
df_government
df_hospital
df_public
df_school
df_sports
df_supermarket
df_toilet
df_transport

In [8]:
df_civic.rename(columns={'InputID':'Building', 'Distance':'Civic'}, inplace = True)
df_fire.rename(columns={'InputID':'Building', 'Distance':'Fire'}, inplace = True)
df_government.rename(columns={'InputID':'Building', 'Distance':'Government'}, inplace = True)
df_hospital.rename(columns={'InputID':'Building', 'Distance':'Hospital'}, inplace = True)
df_public.rename(columns={'InputID':'Building', 'Distance':'Public'}, inplace = True)
df_school.rename(columns={'InputID':'Building', 'Distance':'School'}, inplace = True)
df_sports.rename(columns={'InputID':'Building', 'Distance':'Sports'}, inplace = True)
df_toilet.rename(columns={'InputID':'Building', 'Distance':'Toilet'}, inplace = True)
df_transport.rename(columns={'InputID':'Building', 'Distance':'Transport'}, inplace = True)

In [9]:
df = df_building.merge(df_civic, on='Building', how = 'inner')
df = df.merge(df_fire, on='Building', how = 'inner')
df = df.merge(df_government, on='Building', how = 'inner')
df = df.merge(df_hospital, on='Building', how = 'inner')
df = df.merge(df_public, on='Building', how = 'inner')
df = df.merge(df_school, on='Building', how = 'inner')
df = df.merge(df_sports, on='Building', how = 'inner')
df = df.merge(df_toilet, on='Building', how = 'inner')
df = df.merge(df_transport, on='Building', how = 'inner')

In [10]:
df.columns

Index(['Building', 'name_en', 'building', 'building_l', 'building_m',
       'addr_full', 'addr_house', 'addr_stree', 'addr_city', 'office',
       'source', 'name_zh', 'osm_id', 'osm_type', 'lat', 'lon', 'Name_2',
       'description', 'timestamp', 'begin', 'end', 'altitudeMode',
       'tessellate', 'extrude', 'visibility', 'drawOrder', 'icon', 'id', 'cp',
       'childNum', 'Civic', 'Fire', 'Government', 'Hospital', 'Public',
       'School', 'Sports', 'Toilet', 'Transport'],
      dtype='object')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100663 entries, 0 to 100662
Data columns (total 39 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Building      100663 non-null  object 
 1   name_en       1023 non-null    object 
 2   building      100663 non-null  object 
 3   building_l    16902 non-null   object 
 4   building_m    0 non-null       float64
 5   addr_full     2 non-null       object 
 6   addr_house    1402 non-null    object 
 7   addr_stree    602 non-null     object 
 8   addr_city     463 non-null     object 
 9   office        497 non-null     object 
 10  source        1193 non-null    object 
 11  name_zh       2362 non-null    object 
 12  osm_id        100663 non-null  int64  
 13  osm_type      100663 non-null  object 
 14  lat           100663 non-null  float64
 15  lon           100663 non-null  float64
 16  Name_2        100663 non-null  object 
 17  description   0 non-null       float64
 18  time

In [12]:
df['Civic']

0         [20628.6846754938, 31452.6205282022, 31514.355...
1         [20595.7698531045, 31412.0947527543, 31473.259...
2         [20627.4248578197, 31438.6779288316, 31499.422...
3         [20600.7378457278, 31407.613488466, 31468.0393...
4         [20534.2865295001, 31338.2766479043, 31398.532...
                                ...                        
100658    [52029.2638689452, 62111.1140301271, 62107.933...
100659    [18194.1284939614, 17826.6328741492, 17918.236...
100660    [51832.7157372575, 61893.2254844011, 61888.824...
100661    [51825.1309812148, 61886.1156163727, 61881.745...
100662    [51847.0398221019, 61905.6849241442, 61901.169...
Name: Civic, Length: 100663, dtype: object

In [13]:
df[['Civic', 'Fire', 'Government', 'Hospital', 'Public', 'School', 'Sports', 'Toilet', 'Transport']] = df[['Civic', 'Fire', 'Government', 'Hospital', 'Public', 'School', 'Sports', 'Toilet', 'Transport']].applymap(lambda x: np.divide(x, 10000) if isinstance(x, list) else x)

In [14]:
#df[['Civic', 'Government']] = df[['Civic', 'Government']].applymap(lambda x: np.divide(x, 10000) if isinstance(x, list) else x)

In [15]:
distance_map = {
    'Civic': 1.5,
    'Fire': 1.0,
    'Government': 10.0,
    'Hospital': 10.0,
    'Public': 2.0,
    'School': 1.0,
    'Sports': 1.5,
    'Supermarket': 1.0,
    'Toilet': 0.1,
    'Transport': 2.0,
}

In [16]:
%%time
for facility, max_dist in distance_map.items():
    try:
        df[f'{facility}_nearby'] = df[facility].apply(lambda lst: [d for d in lst if d <= max_dist])
    except:
        pass

CPU times: total: 1.47 s
Wall time: 1.5 s


In [17]:
%%time
for facility, max_dist in distance_map.items():
    try:
        df[f'{facility}_nearby'] = df[f'{facility}_nearby'].apply(lambda lst: [round(max_dist - d, 2) for d in lst])
    except:
        pass


CPU times: total: 11 s
Wall time: 11.3 s


In [18]:
%%time
for facility in distance_map:
    try:
        df[f'{facility}_Score'] = df[f'{facility}_nearby'].apply(sum)
    except:
        pass

CPU times: total: 297 ms
Wall time: 295 ms


In [20]:
from sklearn.preprocessing import MinMaxScaler

# Select score columns
score_cols = [col for col in df.columns if col.endswith('_Score')]

# Initialize scaler with feature range 0-10
scaler = MinMaxScaler(feature_range=(0, 10))

# Fit and transform, then round to 2 decimals
df[score_cols] = pd.DataFrame(
    scaler.fit_transform(df[score_cols]),
    columns=score_cols,
    index=df.index
).round(2)


In [22]:
df.columns

Index(['Building', 'name_en', 'building', 'building_l', 'building_m',
       'addr_full', 'addr_house', 'addr_stree', 'addr_city', 'office',
       'source', 'name_zh', 'osm_id', 'osm_type', 'lat', 'lon', 'Name_2',
       'description', 'timestamp', 'begin', 'end', 'altitudeMode',
       'tessellate', 'extrude', 'visibility', 'drawOrder', 'icon', 'id', 'cp',
       'childNum', 'Civic', 'Fire', 'Government', 'Hospital', 'Public',
       'School', 'Sports', 'Toilet', 'Transport', 'Civic_nearby',
       'Fire_nearby', 'Government_nearby', 'Hospital_nearby', 'Public_nearby',
       'School_nearby', 'Sports_nearby', 'Toilet_nearby', 'Transport_nearby',
       'Civic_Score', 'Fire_Score', 'Government_Score', 'Hospital_Score',
       'Public_Score', 'School_Score', 'Sports_Score', 'Toilet_Score',
       'Transport_Score'],
      dtype='object')

In [23]:
df = df[['Building', 'lat', 'lon',
       'Civic_Score', 'Fire_Score', 'Government_Score', 'Hospital_Score',
       'Public_Score', 'School_Score', 'Sports_Score', 'Toilet_Score',
       'Transport_Score']]

In [24]:
df.to_csv('BeijingScoreRAW1.csv', index = False)

In [15]:
%%time
columns = ['Civic', 'Fire', 'Government', 'Hospital', 'Public', 'School', 'Sports', 'Toilet', 'Transport']

for col in columns:
    df[f'{col}_1km'] = None
    df[f'{col}_5km'] = None
    df[f'{col}_10km'] = None


CPU times: total: 141 ms
Wall time: 115 ms


In [16]:
df

,Building,name_en,building,building_l,building_m,addr_full,addr_house,addr_stree,addr_city,office,...,School_10km,Sports_1km,Sports_5km,Sports_10km,Toilet_1km,Toilet_5km,Toilet_10km,Transport_1km,Transport_5km,Transport_10km
0,BeijingBuilding0,NaN,house,2,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
1,BeijingBuilding1,NaN,house,2,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
2,BeijingBuilding2,NaN,house,2,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
3,BeijingBuilding3,NaN,house,2,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
4,BeijingBuilding4,NaN,house,2,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100658,BeijingBuilding108551,Pinghu Beacon Tower,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
100659,BeijingBuilding108553,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
100660,BeijingBuilding108554,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
100661,BeijingBuilding108555,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None


In [19]:
%%time
# Define columns
columns = ['Civic', 'Fire', 'Government', 'Hospital', 'Public', 
           'School', 'Sports', 'Toilet', 'Transport']

# Define a function to filter numbers based on distance
def filter_distances(row, columns):
    for col in columns:
        row[f'{col}_1km'] = [number for number in row[col] if number <= 1]
        row[f'{col}_5km'] = [number for number in row[col] if number <= 5]
        row[f'{col}_10km'] = [number for number in row[col] if number <= 10]
    return row

# Apply the function across the dataframe
df = df.apply(lambda row: filter_distances(row, columns), axis=1)


CPU times: total: 2min 2s
Wall time: 2min 5s


In [20]:
df

,Building,name_en,building,building_l,building_m,addr_full,addr_house,addr_stree,addr_city,office,...,School_10km,Sports_1km,Sports_5km,Sports_10km,Toilet_1km,Toilet_5km,Toilet_10km,Transport_1km,Transport_5km,Transport_10km
0,BeijingBuilding0,NaN,house,2,NaN,NaN,NaN,NaN,NaN,NaN,...,"[0.131300749096014, 0.13393641060553702, 0.135...","[0.759067609195165, 0.5035494063733691]","[0.759067609195165, 1.12614400889061, 0.503549...","[0.759067609195165, 1.12614400889061, 0.503549...",[],"[1.66066187478947, 2.22126631535815, 3.0205310...","[1.66066187478947, 2.22126631535815, 3.0205310...","[0.274719780547754, 0.27406201470722197, 0.273...","[0.274719780547754, 0.27406201470722197, 0.273...","[0.274719780547754, 0.27406201470722197, 0.273..."
1,BeijingBuilding1,NaN,house,2,NaN,NaN,NaN,NaN,NaN,NaN,...,"[0.126454383314928, 0.129157786353193, 0.13057...","[0.763262207682786, 0.508585167422914]","[0.763262207682786, 1.12343768681073, 0.508585...","[0.763262207682786, 1.12343768681073, 0.508585...",[],"[1.6596673457202502, 2.21938699388618, 3.01870...","[1.6596673457202502, 2.21938699388618, 3.01870...","[0.27785634666888, 0.277158684785473, 0.276327...","[0.27785634666888, 0.277158684785473, 0.276327...","[0.27785634666888, 0.277158684785473, 0.276327..."
2,BeijingBuilding2,NaN,house,2,NaN,NaN,NaN,NaN,NaN,NaN,...,"[0.122274094648953, 0.124945538773291, 0.12634...","[0.760739547926893, 0.512643427567026]","[0.760739547926893, 1.1187460159401301, 0.5126...","[0.760739547926893, 1.1187460159401301, 0.5126...",[],"[1.66387165013058, 2.2232635873183, 3.02260372...","[1.66387165013058, 2.2232635873183, 3.02260372...","[0.274665640431148, 0.27394481493859296, 0.273...","[0.274665640431148, 0.27394481493859296, 0.273...","[0.274665640431148, 0.27394481493859296, 0.273..."
3,BeijingBuilding3,NaN,house,2,NaN,NaN,NaN,NaN,NaN,NaN,...,"[0.119716950123523, 0.12244122058773001, 0.123...","[0.763962500488764, 0.515402174882978]","[0.763962500488764, 1.1176675401476899, 0.5154...","[0.763962500488764, 1.1176675401476899, 0.5154...",[],"[1.66258587940203, 2.22143119118328, 3.0208006...","[1.66258587940203, 2.22143119118328, 3.0208006...","[0.27735719922707297, 0.27661393177125804, 0.2...","[0.27735719922707297, 0.27661393177125804, 0.2...","[0.27735719922707297, 0.27661393177125804, 0.2..."
4,BeijingBuilding4,NaN,house,2,NaN,NaN,NaN,NaN,NaN,NaN,...,"[0.11931253003863801, 0.122139114489735, 0.122...","[0.7709595861437201, 0.516610632014796]","[0.7709595861437201, 1.11968531500791, 0.51661...","[0.7709595861437201, 1.11968531500791, 0.51661...",[],"[1.6572994108357801, 2.21554141816929, 3.01493...","[1.6572994108357801, 2.21554141816929, 3.01493...","[0.28404873236620704, 0.283289459280832, 0.282...","[0.28404873236620704, 0.283289459280832, 0.282...","[0.28404873236620704, 0.283289459280832, 0.282..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100658,BeijingBuilding108551,Pinghu Beacon Tower,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"[1.6187107290276501, 1.6485556306969, 1.609082...","[0.668163416955071, 0.891134093888941]","[1.09396735402331, 1.0688953219605402, 1.66573...","[1.09396735402331, 1.0688953219605402, 1.66573...",[],[4.82653695300782],"[4.82653695300782, 5.39612644838993, 6.1954139...","[0.138004414592435, 0.037181477149124495, 0.11...","[0.138004414592435, 0.037181477149124495, 0.11...","[0.138004414592435, 0.037181477149124495, 0.11..."
100659,BeijingBuilding108553,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"[0.43574956679551796, 0.437163952438011, 0.434...","[0.479982493308485, 0.872016290503434]","[0.479982493308485, 1.29397092027753, 1.295550...","[0.479982493308485, 1.29397092027753, 1.295550...",[],"[3.6396915496283997, 3.14680089140059, 2.87980...","[3.6396915496283997, 3.14680089140059, 2.87980...","[0.524368923683654, 0.573921067536201, 0.63071...","[0.524368923683654, 0.573921067536201, 0.63071...","[0.524368923683654, 0.573921067536201, 0.63071..."
100660,BeijingBuilding108554,NaN,yes,NaN,NaN,NaN,

In [21]:
%%time
# Define columns
columns = ['Civic', 'Fire', 'Government', 'Hospital', 'Public', 
           'School', 'Sports', 'Toilet', 'Transport']

# Define a function to apply element-wise
def transform_list(lst, val):
    return [round(val - x, 2) for x in lst]

# Use applymap to vectorize calculations
for col in columns:
    df[f'{col}_1km'] = df[f'{col}_1km'].map(lambda x: transform_list(x, 1))
    df[f'{col}_5km'] = df[f'{col}_5km'].map(lambda x: transform_list(x, 5))
    df[f'{col}_10km'] = df[f'{col}_10km'].map(lambda x: transform_list(x, 10))



CPU times: total: 2min 5s
Wall time: 2min 21s


In [22]:
%%time
# Define columns
columns = ['Civic', 'Fire', 'Government', 'Hospital', 'Public', 
           'School', 'Sports', 'Toilet', 'Transport']

# Initialize score columns
for col in columns:
    df[f'{col}_1km_Score'] = None
    df[f'{col}_5km_Score'] = None
    df[f'{col}_10km_Score'] = None

# Define a function to calculate scores
def calculate_scores(row, columns):
    for col in columns:
        row[f'{col}_1km_Score'] = sum(row[f'{col}_1km'])
        row[f'{col}_5km_Score'] = sum(row[f'{col}_5km'])
        row[f'{col}_10km_Score'] = sum(row[f'{col}_10km'])
    return row

# Apply the function across the dataframe
df = df.apply(lambda row: calculate_scores(row, columns), axis=1)


CPU times: total: 1min 46s
Wall time: 1min 50s


In [ ]:
df.columns[93]

In [24]:
df

,Building,name_en,building,building_l,building_m,addr_full,addr_house,addr_stree,addr_city,office,...,School_10km_Score,Sports_1km_Score,Sports_5km_Score,Sports_10km_Score,Toilet_1km_Score,Toilet_5km_Score,Toilet_10km_Score,Transport_1km_Score,Transport_5km_Score,Transport_10km_Score
0,BeijingBuilding0,NaN,house,2,NaN,NaN,NaN,NaN,NaN,NaN,...,9.791271,1.510204,8.012048,8.903654,0.0,5.898543,8.027511,6.378270,9.279135,9.387963
1,BeijingBuilding1,NaN,house,2,NaN,NaN,NaN,NaN,NaN,NaN,...,9.797596,1.489796,8.016064,8.905869,0.0,5.898543,8.027511,6.338028,9.271125,9.381163
2,BeijingBuilding2,NaN,house,2,NaN,NaN,NaN,NaN,NaN,NaN,...,9.807084,1.489796,8.020080,8.908084,0.0,5.898543,8.027511,6.378270,9.279135,9.387963
3,BeijingBuilding3,NaN,house,2,NaN,NaN,NaN,NaN,NaN,NaN,...,9.810247,1.469388,8.016064,8.905869,0.0,5.898543,8.027511,6.317907,9.267121,9.377763
4,BeijingBuilding4,NaN,house,2,NaN,NaN,NaN,NaN,NaN,NaN,...,9.810247,1.448980,8.012048,8.903654,0.0,5.903940,8.030106,6.358149,9.275130,9.384563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100658,BeijingBuilding108551,Pinghu Beacon Tower,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.406705,0.897959,7.875502,8.828350,0.0,0.091743,4.728783,9.154930,9.831798,9.857191
100659,BeijingBuilding108553,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.326376,1.326531,7.935743,8.861573,0.0,3.858608,7.046457,4.426559,8.890669,9.058143
100660,BeijingBuilding108554,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.441493,0.938776,7.879518,8.830565,0.0,0.102536,4.749546,9.275654,9.855827,9.877593
100661,BeijingBuilding108555,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.447818,0.938776,7.879518,8.830565,0.0,0.102536,4.749546,9.275654,9.855827,9.877593


In [23]:
import pandas as pd

# Vectorized scaling operation
df.iloc[:, 66:93] = df.iloc[:, 66:93].apply(lambda col: (col - col.min()) / (col.max() - col.min()) * 10, axis=0)


In [25]:
# Optimized approach
try:
    df.iloc[:, 66:93] = df.iloc[:, 66:93].astype(float).round(2)
except Exception as e:
    print('Error:', e)


In [26]:
df.columns

Index(['Building', 'name_en', 'building', 'building_l', 'building_m',
       'addr_full', 'addr_house', 'addr_stree', 'addr_city', 'office',
       'source', 'name_zh', 'osm_id', 'osm_type', 'lat', 'lon', 'Name_2',
       'description', 'timestamp', 'begin', 'end', 'altitudeMode',
       'tessellate', 'extrude', 'visibility', 'drawOrder', 'icon', 'id', 'cp',
       'childNum', 'Civic', 'Fire', 'Government', 'Hospital', 'Public',
       'School', 'Sports', 'Toilet', 'Transport', 'Civic_1km', 'Civic_5km',
       'Civic_10km', 'Fire_1km', 'Fire_5km', 'Fire_10km', 'Government_1km',
       'Government_5km', 'Government_10km', 'Hospital_1km', 'Hospital_5km',
       'Hospital_10km', 'Public_1km', 'Public_5km', 'Public_10km',
       'School_1km', 'School_5km', 'School_10km', 'Sports_1km', 'Sports_5km',
       'Sports_10km', 'Toilet_1km', 'Toilet_5km', 'Toilet_10km',
       'Transport_1km', 'Transport_5km', 'Transport_10km', 'Civic_1km_Score',
       'Civic_5km_Score', 'Civic_10km_Score', 'Fire

In [27]:
df = df[['Building', 'lat', 'lon', 'Civic_1km_Score',
       'Civic_5km_Score', 'Civic_10km_Score', 'Fire_1km_Score',
       'Fire_5km_Score', 'Fire_10km_Score', 'Government_1km_Score',
       'Government_5km_Score', 'Government_10km_Score', 'Hospital_1km_Score',
       'Hospital_5km_Score', 'Hospital_10km_Score', 'Public_1km_Score',
       'Public_5km_Score', 'Public_10km_Score', 'School_1km_Score',
       'School_5km_Score', 'School_10km_Score', 'Sports_1km_Score',
       'Sports_5km_Score', 'Sports_10km_Score', 'Toilet_1km_Score',
       'Toilet_5km_Score', 'Toilet_10km_Score', 'Transport_1km_Score',
       'Transport_5km_Score', 'Transport_10km_Score']]

In [28]:
df

,Building,lat,lon,Civic_1km_Score,Civic_5km_Score,Civic_10km_Score,Fire_1km_Score,Fire_5km_Score,Fire_10km_Score,Government_1km_Score,...,School_10km_Score,Sports_1km_Score,Sports_5km_Score,Sports_10km_Score,Toilet_1km_Score,Toilet_5km_Score,Toilet_10km_Score,Transport_1km_Score,Transport_5km_Score,Transport_10km_Score
0,BeijingBuilding0,40.110081,116.198380,0.0,4.16,6.83,0.0,4.24,7.12,0.0,...,9.79,1.51,8.01,8.90,0.0,5.90,8.03,6.38,9.28,9.39
1,BeijingBuilding1,40.110232,116.199140,0.0,4.16,6.83,0.0,4.24,7.12,0.0,...,9.80,1.49,8.02,8.91,0.0,5.90,8.03,6.34,9.27,9.38
2,BeijingBuilding2,40.110655,116.199178,0.0,4.16,6.83,0.0,4.22,7.11,0.0,...,9.81,1.49,8.02,8.91,0.0,5.90,8.03,6.38,9.28,9.39
3,BeijingBuilding3,40.110691,116.199680,0.0,4.16,6.83,0.0,4.24,7.12,0.0,...,9.81,1.47,8.02,8.91,0.0,5.90,8.03,6.32,9.27,9.38
4,BeijingBuilding4,40.110417,116.200435,0.0,4.18,6.84,0.0,4.24,7.12,0.0,...,9.81,1.45,8.01,8.90,0.0,5.90,8.03,6.36,9.28,9.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100658,BeijingBuilding108551,40.356257,116.008624,0.0,0.00,3.48,0.0,0.00,3.94,0.0,...,7.41,0.90,7.88,8.83,0.0,0.09,4.73,9.15,9.83,9.86
100659,BeijingBuilding108553,39.949320,116.699697,0.0,6.42,8.06,0.0,3.98,6.99,0.0,...,9.33,1.33,7.94,8.86,0.0,3.86,7.05,4.43,8.89,9.06
100660,BeijingBuilding108554,40.356084,116.012264,0.0,0.00,3.50,0.0,0.00,3.96,0.0,...,7.44,0.94,7.88,8.83,0.0,0.10,4.75,9.28,9.86,9.88
100661,BeijingBuilding108555,40.356004,116.012271,0.0,0.00,3.50,0.0,0.00,3.96,0.0,...,7.45,0.94,7.88,8.83,0.0,0.10,4.75,9.28,9.86,9.88


In [29]:
df.to_csv('BeijingScoreRAW.csv', index = False)